# Naive Bayes Classifier

In [135]:
import csv
import random
import math
import numpy as np
import pandas as pd

In [171]:
# titanic survive
df = pd.read_csv('titanic.csv',header=0,usecols=['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Survived'])
df= df.dropna(axis=0)
y=df.iloc[:,0]
print(df.iloc[0,:],'\n')
#replace strings values to numeric
df['Sex'] = df['Sex'].map({'female': 1, 'male': 0})

df['Cherbourg'] =   df['Embarked'].map({'C' : 1, 'Q':0, 'S':0})
df['Queenstown'] =  df['Embarked'].map({'C' : 0, 'Q':1, 'S':0})
df['Southampton'] = df['Embarked'].map({'C' : 0, 'Q':0, 'S':1})

df = df.drop(['Survived','Embarked'], axis=1)
df["survive"]=y

print(df.iloc[0,:])

dataset = df.values
splitRatio = 0.67
print("datatype",type(dataset))
trainSize = int(dataset.shape[0] * splitRatio)


# for i in np.arange(x11.shape[0]):
#     if x11[i,0]=='female':
#         x11[i,0]=0.
#     else:
#         x11[i,0]=1.
        

Survived       0
Pclass         3
Sex         male
Age           22
SibSp          1
Parch          0
Fare        7.25
Embarked       S
Name: 0, dtype: object 

Pclass          3.00
Sex             0.00
Age            22.00
SibSp           1.00
Parch           0.00
Fare            7.25
Cherbourg       0.00
Queenstown      0.00
Southampton     1.00
survive         0.00
Name: 0, dtype: float64
datatype <class 'numpy.ndarray'>


In [186]:
def loadCsv():
    # titanic survive
    df = pd.read_csv('titanic.csv',header=0,usecols=['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Survived'])
    df= df.dropna(axis=0)
    y=df.iloc[:,0]
    #replace strings values to numeric
    df['Sex'] = df['Sex'].map({'female': 1, 'male': 0})

    df['Cherbourg'] =   df['Embarked'].map({'C' : 1, 'Q':0, 'S':0})
    df['Queenstown'] =  df['Embarked'].map({'C' : 0, 'Q':1, 'S':0})
    df['Southampton'] = df['Embarked'].map({'C' : 0, 'Q':0, 'S':1})

    df = df.drop(['Survived','Embarked'], axis=1)
    df["survive"]=y

    dataset = df.values
    return dataset

In [173]:
def splitDataset(dataset, splitRatio):
    # Training set size
    trainSize = int(dataset.shape[0] * splitRatio)
    
    # List of randomly chosen indicies
    indices = np.random.permutation(dataset.shape[0])
    
    # Split indicies for training and test set by trainSize
    training_idx, test_idx = indices[:trainSize], indices[trainSize:]
    
    # Create training and test sets by indicies
    training, test = dataset[training_idx,:], dataset[test_idx,:]
    
    return training, test

In [174]:
def separateByClass(dataset):
    # Here we limit our classes to 0 and 1
    # You need to generalize this for arbitrary number of classes
    ones = dataset[np.where(dataset[:, -1]==1), :]
    zeros = dataset[np.where(dataset[:, -1]==0), :]
    
    p_ones_prob = ones.shape[1] / dataset.shape[0]
    p_zeros_prob = zeros.shape[1] / dataset.shape[0]
    
    return {
        1: (ones, p_ones_prob),
        0: (zeros, p_zeros_prob)
    }

In [175]:
def summarize(dataset):
    # Calculate means and standart deviations with one degree of freedom for each attribute
    # We do it by column which is axis 1
    # Also we remove last elements (guess why?)
    means = dataset[0].mean(axis=1)[0][:-1]
    stds  = dataset[0].std(axis=1, ddof=1)[0][:-1]
    p_c = dataset[1]
    
    # Think what we do here?
    return means, stds, p_c

In [176]:
def summarizeByClass(dataset):
    # Divide dataset by class and summarize it
    separated = separateByClass(dataset)
    
    summaries = {}
    
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    
    return summaries

In [177]:
def calculateProbability(x, mean, stdev):
    # Calculate probability by x, mean and std
    # 1/(sqrt(2pi)*std)*exp(-(x-mean)^2/(2std^2))
    return np.exp(-(x - mean)**2/(2 * stdev**2)) / (np.sqrt(2 * np.pi) * stdev)

In [178]:
def calculateClassProbabilities(summaries, inputVector):
    # Calculate probabilities for input vector from test set
    probabilities = {}
    
    for classValue, classSummaries in summaries.items():
        
        means = classSummaries[0]
        stds  = classSummaries[1]
        p_c   = classSummaries[2]
        
        # Calculate corresonding probabilities and multiply them
        probabilities[classValue] = np.prod(calculateProbability(inputVector[:-1], means, stds)) * p_c

    return probabilities

In [179]:
def predict(summaries, inputVector):
    # Calculate probabilities
    probabilities = calculateClassProbabilities(summaries, inputVector)
    
    # Init values of probability and label
    bestLabel, bestProb = None, -1
    
    # Check probability of which class is better
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    
    return bestLabel

In [180]:
def getPredictions(summaries, testSet):
    # For each probability find optimal labels
    predictions = []
    
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)

    return predictions

In [181]:
def getAccuracy(testSet, predictions):
    # Check accuracy
    correct = 0
    
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [200]:
def main():
    
    # Set split ratio
    splitRatio = 0.09
    
    # Load dataset and return numpy array
    dataset = loadCsv()
    
    # Split dataset
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    
    # Log row amounts
    print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
    
    # Prepare model
    summaries = summarizeByClass(trainingSet)
    
    # Test model
    predictions = getPredictions(summaries, testSet)
    
    accuracy = getAccuracy(testSet, predictions)
    
    print('Accuracy: {0}%'.format(accuracy))

In [201]:
main()

Split 712 rows into train=64 and test=648 rows
Accuracy: 76.08024691358025%
